In [2]:
import pandas as pd
import os
import sys
import re
import statsmodels.api as sm
import seaborn as sns
import pmdarima as pm
import numpy as np

from pathlib import Path
from matplotlib import pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from sktime.forecasting.arima import AutoARIMA

from src.import_downcasting import import_downcasting
# from src.utils import gen_weekday_num, ts_sales_plot, adf_test_autolag
from src.models.timeseries.unitrootTest import searchStationarySeriesADF
from src.models.timeseries.autoARIMA import autoARIMA

In [3]:
# root = os.path.join(Path(Path(os.getcwd()).parent.absolute()), 'assets/data')
root = os.path.join('assets/data')


calendar = import_downcasting(os.path.join(root, "raw/calendar.csv"))
# sales = import_downcasting(os.path.join(root, "sales_train_evaluation.csv"))
sell_price = import_downcasting(os.path.join(root, "raw/sell_prices.csv"))

# sales_ca1 = sales[sales.store_id == "CA_1"]
# sales_ca1.to_csv(os.path.join(root, "sales_ca1.csv"), index=False)
# del sales

In [4]:
sales_ca1 = import_downcasting(os.path.join(root, "sales_ca1.csv"))
sales_ca1 = pd.melt(sales_ca1, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name="date_code", value_name ="sales")
sales_ca1 = pd.merge(left=sales_ca1, right=calendar, left_on='date_code', right_on='d', how='left')
sales_ca1 = pd.merge(left=sales_ca1, right=sell_price, left_on=['store_id', 'item_id', 'wm_yr_wk'], right_on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
# sales_ca1['weekday_num'] = sales_ca1['weekday'].apply(gen_weekday_num)

In [5]:
# path = os.path.join(root, 'sales_ca1_melted.csv')
# model = autoARIMA.fit(root,path)
# # model.train(root, 'root/sales_ca1_melted.csv')

AttributeError: type object 'autoARIMA' has no attribute 'fit'

In [7]:
ranked_item_unit = sales_ca1.groupby(['item_id'])['sales'].sum().sort_values(ascending=False).reset_index()
sales_ca1_rank1 = sales_ca1[sales_ca1.item_id == 'FOODS_3_090'].reset_index(drop=True)
sales_ca1_rank1 = sales_ca1_rank1.set_index('date')

In [8]:
series = sales_ca1_rank1['sales']
series_2 = series.diff(1)

series.head()
print(series_2.head())
print(series.diff(0).head())
print(sales_ca1_rank1.info())

date
2011-01-29      NaN
2011-01-30     75.0
2011-01-31   -135.0
2011-02-01      0.0
2011-02-02     15.0
Name: sales, dtype: float32
date
2011-01-29    0.0
2011-01-30    0.0
2011-01-31    0.0
2011-02-01    0.0
2011-02-02    0.0
Name: sales, dtype: float32
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1941 entries, 2011-01-29 to 2016-05-22
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id            1941 non-null   category
 1   item_id       1941 non-null   category
 2   dept_id       1941 non-null   category
 3   cat_id        1941 non-null   category
 4   store_id      1941 non-null   object  
 5   state_id      1941 non-null   category
 6   date_code     1941 non-null   object  
 7   sales         1941 non-null   int16   
 8   wm_yr_wk      1941 non-null   int16   
 9   weekday       1941 non-null   category
 10  wday          1941 non-null   int8    
 11  month         1941 non-null   int8    
 12

In [9]:
sationary = searchStationarySeriesADF(sales_ca1_rank1['sales'])
diff_order = sationary.get_diff_order_stationary()

In [10]:
model = AutoARIMA(
                      start_p=1, 
                      start_q=1,
                      max_p=30, 
                      max_q=30, # maximum p and q
                      d=diff_order,
                      max_d=diff_order,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore', 
                      suppress_warnings=True, 
                      stepwise=True,
                      alpha=0.05)

model.fit(sales_ca1_rank1['sales'])


Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=19748.820, Time=0.26 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=21240.359, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=19772.978, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=20326.482, Time=0.18 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=22883.216, Time=0.01 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=19596.266, Time=0.64 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=19757.711, Time=0.08 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=19581.651, Time=1.02 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=19739.283, Time=0.22 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept   : AIC=19560.794, Time=1.08 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=19708.413, Time=0.32 sec
 ARIMA(5,0,1)(0,0,0)[0] intercept   : AIC=19531.545, Time=1.38 sec
 ARIMA(5,0,0)(0,0,0)[0] intercept   : AIC=19643.973, Time=0.53 sec
 ARIMA(6,0,1)(0,0,0)[0] intercept   : AIC=19498.105, Time=1.45 sec
 ARIMA(6,0,0)(0,0,0

AutoARIMA(D=0, d=0, error_action='ignore', max_d=0, max_p=30, max_q=30,
          seasonal=False, start_P=0, start_p=1, start_q=1,
          suppress_warnings=True, trace=True)

In [13]:
# model.fit(sales_ca1_rank1['sales'])
model.get_fitted_params()["order"]
print(type(model.get_fitted_params()["order"]))


<class 'tuple'>


In [ ]:
sns.boxplot(x="weekday_num", y='sales', data=sales_ca1_rank1, order=[1,2,3,4,5,6,7])

ValueError: Could not interpret input 'weekday_num'

In [ ]:
result = seasonal_decompose(sales_ca1_rank1['sales'], model='additive', period=10)

fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(15, 12)


# month_year = sales_ca1_rank1.groupby(['year', 'month']).sales.sum().reset_index()
# month_year = month_year[month_year.year > 2013].reset_index()
# sns.barplot(data=month_year, x=month_year.index, y='sales')

In [ ]:
sales_ca1_rank1['sales_first_diff'] = sales_ca1_rank1['sales'] - sales_ca1_rank1['sales'].shift(1)
sales_ca1_rank1 = sales_ca1_rank1.loc[sales_ca1_rank1['sales_first_diff'].notnull()]

In [ ]:
ts_sales_plot(sales_ca1_rank1, 'sales')
ts_sales_plot(sales_ca1_rank1, 'sales_first_diff')


In [ ]:
rest = adfuller(sales_ca1_rank1['sales'])
print(rest)
print(type(rest))

In [ ]:
searchStationarySeriesADF(sales_ca1_rank1['sales'])

In [ ]:
sm.graphics.tsa.plot_acf(sales_ca1_rank1.sales, lags=40)
sm.graphics.tsa.plot_pacf(sales_ca1_rank1.sales, lags=40) 

In [ ]:
sm.graphics.tsa.plot_acf(sales_ca1_rank1.sales_first_diff, lags=40) 
sm.graphics.tsa.plot_pacf(sales_ca1_rank1.sales_first_diff, lags=40) 